# Time series predictor

In [2]:
import numpy as np
import time
from datetime import date
import datetime
from datetime import timedelta  
import csv
import holidays # for importing the public holidays
import re

bologna_holidays = holidays.Italy(prov='BO') # Get the holidays in Bologna, Italy :)

In [3]:
# Variables
data_dir = "data"
dataset = "data/LastSeenDataset.csv"

- Feature extraction: we first extract the features given the time series data of Telegram accesses.
- Supposition: last Telegram access in very similar to the time the person goes to sleep

## Feature engineering
Possible features to extract: 
1. Last seen time (arguably the most important)
2. Wake up time
3. Number of Telegram accesses during the previous day
4. Day of the week
5. Public holiday presence in the following day (using the holidays library)
6. (time spent on Telegram)


In [4]:
## OLD FUNCTIONS
'''
We don't need these functions since we have already converte
def date_time_separate(data):
    for i in range(len(data)):
        d = (re.split(' ', str(data[i][0]))[0])
        #d = d.split('-')
        t = (re.split(' ', str(data[i][0]))[1])
        #t = t.split(':')
        data[i] = [d, t]
    return data

def singe_value_split(data):
    for i in range(len(data)):
        d = (re.split(' ', str(data[i][0]))[0])
        d = d.split('-')
        t = (re.split(' ', str(data[i][0]))[1])
        t = t.split(':')
        data[i] = [d, t]
    return data
'''


"\nWe don't need these functions since we have already converte\ndef date_time_separate(data):\n    for i in range(len(data)):\n        d = (re.split(' ', str(data[i][0]))[0])\n        #d = d.split('-')\n        t = (re.split(' ', str(data[i][0]))[1])\n        #t = t.split(':')\n        data[i] = [d, t]\n    return data\n\ndef singe_value_split(data):\n    for i in range(len(data)):\n        d = (re.split(' ', str(data[i][0]))[0])\n        d = d.split('-')\n        t = (re.split(' ', str(data[i][0]))[1])\n        t = t.split(':')\n        data[i] = [d, t]\n    return data\n"

In [30]:
with open(dataset, newline='') as csvfile:
    date_list = list(csv.reader(csvfile))

# Transform all the elements in the dataset into date objects and sort them
def convert_to_dates(dates):
    for i in range(len(dates)):
        dates[i][0] = datetime.datetime.strptime(dates[i][0], "%Y-%m-%d %H:%M:%S" ) 
    return sorted(dates)

date_list = convert_to_dates(date_list)

print(date_list[0][0] in bologna_holidays)

print(date_list[0][0].year)
#print(date_list)

False
2020


In [57]:
from statistics import mean ## needed for avoiding outliers

# Detect last seen time at night

fall_asleep_datetime = []
wake_up_datetime = []

fall_asleep = [] # array containing seconds from MIN_XX time to go to bed
time_slept = []
day_of_week = []
holiday_presence = []
num_accesses = []

# Last seen during this time will be considered as last seen time
# BEWARE: cannot detect times out of this range
MIN_H, MIN_M, MIN_S = 21, 0, 0
MAX_H, MAX_M, MAX_S = 5, 0, 0
MAX_SLEPT = 12 # In hours
MAX_ACCESSES = 100
NUM_FEATURES = 5 # features we choose to use

## Suppose detection started before MIN_HOUR
# Initialize
min_asleep_time = date_list[0][0]
min_asleep_time = min_asleep_time.replace(hour= MIN_H, minute= MIN_M, second= MIN_S)
max_asleep_time = date_list[0][0]
max_asleep_time = max_asleep_time.replace(day= date_list[0][0].day, hour= MAX_H, minute= MAX_M, second= MAX_S) + timedelta(days=1)
# max_t = min_asleep_time
# m_fall_asleep_datetime_time.hour, min_asleep_time.min, min_asleep_time.second = MIN_H, MIN_M, MIN_S

saved = True
n_accesses = 0 # initialize to 0 the counter for number of status changes

# Measure relative sleeptime in seconds
def to_time_sec(sleeptime):
    return (sleeptime - min_asleep_time).total_seconds()# (wrong?) + 86400 # Adding seconds in a day ('cause day before))

# Measure time difference in seconds
def time_diff_sec(t_f, t_0):
    return (t_f - t_0).total_seconds()

def fill_outlier(sleeptime):
    """Replace data by their averages"""
    fall_asleep_datetime.append(fall_asleep_datetime[-1] + timedelta(days=1)) # Add same sleeptime as yesterday with + 1 day
    wake_up_datetime.append(wake_up_datetime[-1] + timedelta(days=1)) # Add same wakeup as yesterday with + 1 day
    fall_asleep.append(mean(fall_asleep))
    time_slept.append(mean(time_slept))
    day_of_week.append(fall_asleep_datetime[-1].weekday())
    holiday_presence.append(fall_asleep_datetime[-1] in bologna_holidays)
    num_accesses.append(mean(num_accesses))
    
def save_data(d, n_accesses, sleeptime):
    # We save all the data in one single time so not to create mismatches
    '''
    " Fill in the blanks "
    Let's check if there are no new data: it means that the data was either not collected,
    or it is an outlier: hence, let's set the new sleeptime to be the same as the last recorded one
    '''
    sleeptime_s = to_time_sec(sleeptime)
#     print(sleeptime)
    time_slept_s = time_diff_sec(d, sleeptime)
    
    fall_asleep_datetime.append(sleeptime)
    wake_up_datetime.append(d)
    fall_asleep.append(sleeptime_s)
    time_slept.append(time_slept_s)
    day_of_week.append(d.weekday())
    holiday_presence.append(d in bologna_holidays)
    num_accesses.append(n_accesses)

for i in range(len(date_list)):
    # If time is a candidate for falling asleep (=good night) then save it
    d = date_list[i][0]
    if d > min_asleep_time and d < max_asleep_time:
        sleeptime = d # save this time
        saved = False
    else:
        if len(fall_asleep_datetime) == 0:
            if not saved:
                save_data(d, n_accesses, sleeptime) # First datum is not empty, of course
        else:
            if (d - wake_up_datetime[-1] > timedelta(days=1, hours = MAX_SLEPT)):
                fill_outlier(sleeptime)
                d = (fall_asleep_datetime[-1] + timedelta(days=1))
            if not saved:
                save_data(d, n_accesses, sleeptime)
        # Update indexes every first Telegram access in the morning
        min_asleep_time = d.replace(hour= MIN_H, minute= MIN_M, second= MIN_S)
        max_asleep_time = d.replace(day= d.day, hour= MAX_H, minute= MAX_M, second= MAX_S) + timedelta(days=1)
        saved = True 
        n_accesses = 0

    n_accesses += 1 # increase counter for every different last seen ~ number of accesses
            
print("Fall asleep time: ", fall_asleep_datetime)
print("Wake up time: ", wake_up_datetime)

# Data actually used for learning
print("Fall asleep time (seconds from MIN): ", fall_asleep)
print("Time slept (seconds): ", time_slept)
print("Day of the week (from 0 = Monday): ", day_of_week)
print("Holiday presence: ", holiday_presence)
print("Number of accesses: ", num_accesses)

Fall asleep time:  [datetime.datetime(2020, 10, 18, 1, 48, 7), datetime.datetime(2020, 10, 19, 1, 20, 51), datetime.datetime(2020, 10, 20, 0, 34, 19), datetime.datetime(2020, 10, 21, 1, 1, 35), datetime.datetime(2020, 10, 22, 0, 54, 36), datetime.datetime(2020, 10, 23, 1, 18, 14), datetime.datetime(2020, 10, 24, 1, 18, 14), datetime.datetime(2020, 10, 25, 2, 20, 39), datetime.datetime(2020, 10, 25, 22, 35, 4), datetime.datetime(2020, 10, 27, 1, 20, 5), datetime.datetime(2020, 10, 28, 1, 42, 46), datetime.datetime(2020, 10, 29, 1, 11, 3), datetime.datetime(2020, 10, 30, 1, 42, 10), datetime.datetime(2020, 10, 30, 23, 17, 47), datetime.datetime(2020, 10, 31, 22, 14, 9), datetime.datetime(2020, 11, 1, 23, 47, 12), datetime.datetime(2020, 11, 2, 21, 48, 19), datetime.datetime(2020, 11, 4, 2, 21, 20), datetime.datetime(2020, 11, 5, 1, 4, 23), datetime.datetime(2020, 11, 6, 0, 55, 28), datetime.datetime(2020, 11, 7, 1, 52, 29)]
Wake up time:  [datetime.datetime(2020, 10, 18, 9, 48, 7), datet

In [56]:
# Data preparation for the model without PyTorch
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

def to_array(data):
    return np.array(data).reshape(-1, 1)

# We preprocess the data so that they are "normalized" to a fixed scale (for easy time recovery)
def build_matrix(asleep, slept, day, holiday, num_accesses):
    num_observations = len(asleep)
    # Transform all the arrays into tensors
    #scaler = MinMaxScaler(feature_range=(0, (MAX_H+24-MIN_H)*3600 + (MAX_M-MIN_M)*60 + (MAX_S-MIN_S)))
    asleep = to_array(asleep) /  ((MAX_H+24-MIN_H)*3600 + (MAX_M-MIN_M)*60 + (MAX_S-MIN_S))
    slept = to_array(slept) / (MAX_SLEPT*3600)
    day = to_array(day) / 6
    holiday = to_array(holiday) # no need to normalize,  true/false
    accesses = to_array(num_accesses)/ MAX_ACCESSES
    return np.concatenate((asleep, slept, day, holiday, accesses)).reshape(NUM_FEATURES, num_observations)

# Matrix with dimensions m x n ; m = num_features, n = num_observations
data_matrix = build_matrix(fall_asleep, time_slept, day_of_week, holiday_presence, num_accesses)
print(data_matrix)


[[ 0.60024306  0.5434375   0.44649306  0.50329861  0.48875     0.53798611
   0.52003472  0.66802083  0.19805556  0.54184028  0.58909722  0.52302083
   0.58784722  0.28704861  0.15447917  0.34833333  0.10065972  0.66944444
   0.50913194  0.49055556  0.60934028]
 [ 0.66666667  0.59907407  0.66525463  0.64453704  0.78011574  0.689375
   0.67417052 -0.08668981  1.00483796  0.62780093  0.71046296  0.69877315
   0.63844907  0.84071759  0.91458333  0.78622685  1.17833333  0.40328704
   0.67229167  0.69881944  0.60335648]
 [ 1.          0.          0.16666667  0.33333333  0.5         0.66666667
   0.83333333  1.          0.          0.16666667  0.33333333  0.5
   0.66666667  0.83333333  1.          0.          0.16666667  0.33333333
   0.5         0.66666667  0.83333333]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          0

In [372]:
# Transformation to PyTorch tensor and data augmentation
data_tensor = torch.from_numpy(data_matrix)
print("Data:", data_tensor)

# Data augmentation
# We use the "last 3" trend
# Credits: https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/
train_window = 2

def create_inout_sequences(dt, tw):
    inout_seq = []
    L = dt.shape[1]
    for i in range(L-tw):
        train_seq = torch.zeros(NUM_FEATURES, tw)
        for j in range(NUM_FEATURES):
            train_seq[j]= dt[j][i:i+tw]
        train_label = dt[0][i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    print(inout_seq)
    return inout_seq

train_inout_seq = create_inout_sequences(data_tensor, train_window)


Data: tensor([[0.6002, 0.5434, 0.4465, 0.5033],
        [0.8000, 0.7189, 0.7983, 0.7734],
        [1.0000, 0.0000, 0.1667, 0.3333],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.5600, 0.1500, 0.5600, 0.1700]], dtype=torch.float64)
[(tensor([[0.6002, 0.5434],
        [0.8000, 0.7189],
        [1.0000, 0.0000],
        [0.0000, 0.0000],
        [0.5600, 0.1500]]), tensor([0.4465], dtype=torch.float64)), (tensor([[0.5434, 0.4465],
        [0.7189, 0.7983],
        [0.0000, 0.1667],
        [0.0000, 0.0000],
        [0.1500, 0.5600]]), tensor([0.5033], dtype=torch.float64))]


## Model
- Time series data, so possible idea(s):
    - LSTM

In [373]:
from torch import nn

class LSTM(nn.Module):
    def __init__(self, input_size=NUM_FEATURES, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

LSTM(
  (lstm): LSTM(5, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [374]:
epochs = 150

for i in range(epochs):
    for seq, label in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, label)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

RuntimeError: input.size(-1) must be equal to input_size. Expected 5, got 2